In [1]:
# Tidal Gauge comparison

In [5]:
%load_ext autoreload
%autoreload 2
%config IPCompleter.greedy=True

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
import os
import numpy as np
import netCDF4 as nc
import xarray as xr
import datatree as dt
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from sithom.plot import plot_defaults, label_subplots, lim
from tcpips.constants import DATA_PATH, FIGURE_PATH
from adforce.mesh import xr_loader
plot_defaults()

ImportError: cannot import name 'BoundingBox' from 'sithom.plot' (/mnt/lustre/a2fs-work1/work/n01/n01/sithom/adcirc-swan/sithom/sithom/plot.py)